In [2]:
from ultralytics import YOLO
import cv2
import pandas as pd
import gc
import easyocr
import numpy as np
import csv
from scipy.interpolate import interp1d


from LicensePlate import LicensePlateDetection, LicensePlateReader
from Vehicle import VehicleDetection
from utils import PlateToVehicle, interpolate_bounding_boxes
import matplotlib.pyplot as plt

In [ ]:
plate_det_model_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/models/YOLOv8/yolov8n_90e_cust/runs/detect/train4/weights/best.pt'
testvideo_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/video_data/testvideo_short.mp4'

# OOP
df_cols = ['fr_number','v_id', 'v_bbox','lp_bbox','lp_bbox_score','lp','lp_score']
df_rows = []
# load using Sort
video_capture = cv2.VideoCapture(testvideo_path)

vehicle_detection = VehicleDetection('YOLOv8')
lp_detection = LicensePlateDetection('yolov8n_90e_cust')
lp_reader = LicensePlateReader('easyocr')

#mot_trckr = Sort()
#results = {}
ret = True
frame_indexer=0
while ret:
    if (video_capture.isOpened()== False):
        print("Error opening video stream or file")
    
    ret, frame = video_capture.read()
    
    if ret:
        #if frame_indexer>10:
        #    break
        #results[frame_indexer] = {}
        #det
        det_objs = vehicle_detection.Detect(frame)
        vehicles=[]
        for det_obj in det_objs.boxes.data.tolist():
            obj_x1, obj_y1, obj_x2, obj_y2, obj_score, obj_class_id = det_obj
            if int(obj_class_id) in vehicle_detection.vehicle_ids:
                vehicles.append([obj_x1, obj_y1, obj_x2, obj_y2, obj_score])
                #print(obj_class_id)
        
        track_ids = 0#mot_trckr.update(np.asarray(vehicles))
        #print(track_ids)
        #plates=[]
        det_lps = lp_detection.Detect(frame)
        for indexer, det_lp in enumerate(det_lps.boxes.data.tolist()):
            lp_x1, lp_y1, lp_x2, lp_y2, lp_score, lp_class_id = det_lp
            
            #assign license plate to vehicle
            vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2,vehicle_id = PlateToVehicle(det_lp,track_ids)
            if vehicle_id is not -1:
                #crop
                license_plate = frame[int(lp_y1):int(lp_y2),int(lp_x1):int(lp_x2),:]

                cv2.imwrite(f'/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/dataset/video_licenseplates/{indexer}.jpg', license_plate)

In [25]:
lp_reader = LicensePlateReader('easyocr')
import cv2

lp = cv2.imread('/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/dataset/video_licenseplates/3.jpg')
image = cv2.cvtColor(lp, cv2.COLOR_BGR2GRAY)
sharpening_filter = np.array([[-1, -1, -1],
                              [-1,  9, -1],
                              [-1, -1, -1]])
sharp_image = cv2.filter2D(image, -1, sharpening_filter)
#sharp_image = cv2.filter2D(sharp_image, -1, sharpening_filter)
retval,lp_thresholded = cv2.threshold(sharp_image,34,255,cv2.THRESH_BINARY_INV)
inv_thresholded = cv2.bitwise_not(lp_thresholded)
median_filtered = cv2.medianBlur(inv_thresholded, 3)
gauss3_filtered = cv2.GaussianBlur(inv_thresholded, (3,3), 2)
gauss2_filtered = cv2.GaussianBlur(gauss3_filtered, (3,3), 2)
gauss_filtered = cv2.GaussianBlur(gauss2_filtered, (3,3), 2)
final = cv2.filter2D(gauss_filtered, -1, sharpening_filter)


text, score, img = lp_reader.Read(lp)
print('original: ',text, score)

text, score, img = lp_reader.Read(sharp_image)
print('sharpened: ',text, score)

text, score, img = lp_reader.Read(lp_thresholded)
print('thresholded:',text, score)

text, score, img = lp_reader.Read(inv_thresholded)
print('inv. thresholded:',text, score)

text, score, img = lp_reader.Read(median_filtered)
print('median: ',text, score)

text, score, img = lp_reader.Read(gauss_filtered)
print('gaussian: ',text, score)

cv2.imshow('lp',lp)
cv2.waitKey(0)
cv2.imshow('sharp',sharp_image)
cv2.waitKey(0)
cv2.imshow('threshed',lp_thresholded)
cv2.waitKey(0)
cv2.imshow('inv_threshed',inv_thresholded)
cv2.waitKey(0)
cv2.imshow('median',median_filtered)
cv2.waitKey(0)
cv2.imshow('final',final)
cv2.waitKey(0)

Using CPU. Note: This module is much faster with a GPU.


original:  GJO6EPD 0.3774945997021085
sharpened:  LJDEEAD 0.10828740001885445
thresholded: GJDEEPD 0.6414579814310568
inv. thresholded: 6JQGEPD 0.157182212043082
median:  GJQ6EPD 0.34503582532963706
gaussian:  GJO6EPD 0.14858900646714243


32

In [ ]:
def MultiplyPrepAndReadINPLACE(license_plate):
        formatted_text = None
        results_list = []
        lp_gray = cv2.cvtColor(license_plate,cv2.COLOR_BGR2GRAY)
        sharpening_filter = np.array([[-1, -1, -1],
                              [-1,  9, -1],
                              [-1, -1, -1]])
        sharp_image = cv2.filter2D(lp_gray, -1, sharpening_filter)
        retval,lp_thresholded = cv2.threshold(sharp_image,34,255,cv2.THRESH_BINARY_INV)

        detections = self.model.readtext(lp_thresholded)
        for det in detections:
            bbox, text, score = det
            formatted_text = text.upper().replace(' ','')
        if formatted_text is not None:
            results_list.append((formatted_text,score))

        inv_thresholded = cv2.bitwise_not(lp_thresholded)
        
        detections = self.model.readtext(inv_thresholded)
        for det in detections:
            bbox, text, score = det
            formatted_text = text.upper().replace(' ','')
        if formatted_text is not None:
            results_list.append((formatted_text,score))


        median_filtered = cv2.medianBlur(inv_thresholded, 3)
        gauss3_filtered = cv2.GaussianBlur(inv_thresholded, (3,3), 2)
        gauss2_filtered = cv2.GaussianBlur(gauss3_filtered, (3,3), 2)
        gauss_filtered = cv2.GaussianBlur(gauss2_filtered, (3,3), 2)

        detections = self.model.readtext(gauss_filtered)
        for det in detections:
            bbox, text, score = det
            formatted_text = text.upper().replace(' ','')
        if formatted_text is not None:
            results_list.append((formatted_text,score))
            
        try:
            formatted_text = max(results_list, key=lambda x: x[1])[0]
            score = max(results_list, key=lambda x: x[1])[1]
            # Additional code if needed
        except ValueError:
            print("Error: results_list is empty or contains tuples without a second element.")
            formatted_text = None
            score = None
            # Handle the exception as needed

        return formatted_text, score, gauss_filtered
        

In [21]:
lp_reader = LicensePlateReader('easyocr')
import cv2

lp = cv2.imread('/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/dataset/video_licenseplates/3.jpg')
image = cv2.cvtColor(lp, cv2.COLOR_BGR2GRAY)
sharpening_filter = np.array([[-1, -1, -1],
                              [-1,  9, -1],
                              [-1, -1, -1]])
sharp_image = cv2.filter2D(image, -1, sharpening_filter)
#sharp_image = cv2.filter2D(sharp_image, -1, sharpening_filter)
retval,lp_thresholded = cv2.threshold(sharp_image,44,255,cv2.THRESH_BINARY_INV)
median_filtered = cv2.medianBlur(lp_thresholded, 3)
inv_thresholded = cv2.bitwise_not(median_filtered)
gauss_filtered = cv2.GaussianBlur(inv_thresholded, (3,3), 2)

text, score, img = lp_reader.Read(lp)
print('original: ',text, score)

text, score, img = lp_reader.Read(sharp_image)
print('sharpened: ',text, score)

text, score, img = lp_reader.Read(lp_thresholded)
print('thresholded:',text, score)

text, score, img = lp_reader.Read(median_filtered)
print('median: ',text, score)

text, score, img = lp_reader.Read(inv_thresholded)
print('inv. thresholded:',text, score)

text, score, img = lp_reader.Read(gauss_filtered)
print('gaussian: ',text, score)

cv2.imshow('lp',lp)
cv2.waitKey(0)
cv2.imshow('sharp',sharp_image)
cv2.waitKey(0)
cv2.imshow('threshed',lp_thresholded)
cv2.waitKey(0)
cv2.imshow('median',median_filtered)
cv2.waitKey(0)
cv2.imshow('inv_threshed',inv_thresholded)
cv2.waitKey(0)
cv2.imshow('gauss',gauss_filtered)
cv2.waitKey(0)

Using CPU. Note: This module is much faster with a GPU.


original:  GJO6EPD 0.3774945997021085
sharpened:  LJDEEAD 0.10828740001885445
thresholded: GJDBEPD 0.19982547219625899
median:  GJO6EPD 0.769778708353724
inv. thresholded: "GJOGEPD 0.3447133467574065
gaussian:  "GJO6EPD 0.2046682469103598


32

In [23]:
lp_reader = LicensePlateReader('easyocr')
import cv2

lp = cv2.imread('/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/dataset/video_licenseplates/3.jpg')
image = cv2.cvtColor(lp, cv2.COLOR_BGR2GRAY)
sharpening_filter = np.array([[-1, -1, -1],
                              [-1,  9, -1],
                              [-1, -1, -1]])
sharp_image = cv2.filter2D(image, -1, sharpening_filter)
retval,lp_thresholded = cv2.threshold(sharp_image,44,255,cv2.THRESH_BINARY_INV)
median_filtered = cv2.medianBlur(lp_thresholded, 3)
gauss_filtered = cv2.GaussianBlur(median_filtered, (3,3), 2)
inv_thresholded = cv2.bitwise_not(gauss_filtered)

text, score, img = lp_reader.Read(lp)
print('original: ',text, score)

text, score, img = lp_reader.Read(sharp_image)
print('sharpened: ',text, score)

text, score, img = lp_reader.Read(lp_thresholded)
print('thresholded:',text, score)

text, score, img = lp_reader.Read(median_filtered)
print('median: ',text, score)

text, score, img = lp_reader.Read(gauss_filtered)
print('gaussian: ',text, score)

text, score, img = lp_reader.Read(inv_thresholded)
print('inv. thresholded:',text, score)

cv2.imshow('lp',lp)
cv2.waitKey(0)
cv2.imshow('sharp',sharp_image)
cv2.waitKey(0)
cv2.imshow('threshed',lp_thresholded)
cv2.waitKey(0)
cv2.imshow('median',median_filtered)
cv2.waitKey(0)
cv2.imshow('gauss',gauss_filtered)
cv2.waitKey(0)
cv2.imshow('inv_threshed',inv_thresholded)
cv2.waitKey(0)

Using CPU. Note: This module is much faster with a GPU.


original:  GJO6EPD 0.3774945997021085
sharpened:  LJDEEAD 0.10828740001885445
thresholded: GJDBEPD 0.19982547219625899
median:  GJO6EPD 0.769778708353724
gaussian:  "CJOBEP 0.26017091235484
inv. thresholded: "GJO6EPD 0.2046682469103598


32

In [29]:
instlist = []
instlist.append(('a',0.55))
instlist.append(('b',0.5))
instlist.append(('c',0.7))

print(instlist)

max(instlist, key=lambda x: x[1])[0]

[('a', 0.55), ('b', 0.5), ('c', 0.7)]


'c'

In [4]:
plate_det_model_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/models/YOLOv8/yolov8n_90e_cust/runs/detect/train4/weights/best.pt'
testvideo_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/video_data/testvideo_short.mp4'

# OOP
df_cols = ['fr_number','v_id', 'v_bbox','lp_bbox','lp_bbox_score','lp','lp_score']
df_rows = []
# load using Sort
video_capture = cv2.VideoCapture(testvideo_path)

vehicle_detection = VehicleDetection('YOLOv8')
lp_detection = LicensePlateDetection('yolov8n_90e_cust')
lp_reader = LicensePlateReader('easyocr')

mot_trckr = Sort()
#results = {}
ret = True
frame_indexer=0
while ret:
    if (video_capture.isOpened()== False):
        print("Error opening video stream or file")
    
    ret, frame = video_capture.read()
    
    if ret:
        #if frame_indexer>10:
        #    break
        #results[frame_indexer] = {}
        #det
        det_objs = vehicle_detection.Detect(frame)
        vehicles=[]
        for det_obj in det_objs.boxes.data.tolist():
            obj_x1, obj_y1, obj_x2, obj_y2, obj_score, obj_class_id = det_obj
            if int(obj_class_id) in vehicle_detection.vehicle_ids:
                vehicles.append([obj_x1, obj_y1, obj_x2, obj_y2, obj_score])
                #print(obj_class_id)
        
        track_ids = mot_trckr.update(np.asarray(vehicles))
        #print(track_ids)
        #plates=[]
        det_lps = lp_detection.Detect(frame)
        for indexer, det_lp in enumerate(det_lps.boxes.data.tolist()):
            lp_x1, lp_y1, lp_x2, lp_y2, lp_score, lp_class_id = det_lp
            
            #assign license plate to vehicle
            vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2,vehicle_id = PlateToVehicle(det_lp,track_ids)
            if vehicle_id is not -1:
                #crop
                license_plate = frame[int(lp_y1):int(lp_y2),int(lp_x1):int(lp_x2),:]
                lp_text, lp_text_confscore, lp_prep = lp_reader.PrepAndRead(license_plate)
                print(lp_text, lp_text_confscore)
                

<>:46: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:46: SyntaxWarning: "is not" with a literal. Did you mean "!="?
[ WARN:0@346.541] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@346.542] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin1 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@346.542] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start p

AK6 0.24160493910312653


0: 480x640 1 license, 96.6ms
Speed: 2.8ms preprocess, 96.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 16 cars, 2 buss, 1 truck, 114.8ms
Speed: 3.5ms preprocess, 114.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



IK6 0.3802231069475235


0: 480x640 1 license, 104.0ms
Speed: 3.7ms preprocess, 104.0ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 13 cars, 2 buss, 1 truck, 113.2ms
Speed: 2.9ms preprocess, 113.2ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)



...

0: 480x640 2 licenses, 115.9ms
Speed: 4.0ms preprocess, 115.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



AK6' 0.18284972012043
EFIODZ7 0.3091464095046897


0: 480x640 14 cars, 3 buss, 2 trucks, 116.2ms
Speed: 3.0ms preprocess, 116.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 licenses, 183.4ms
Speed: 3.8ms preprocess, 183.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
/var/folders/ms/lx6k291d71z8f387pwx9g8cm0000gn/T/ipykernel_3561/1800999524.py:46: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if vehicle_id is not -1:
/var/folders/ms/lx6k291d71z8f387pwx9g8cm0000gn/T/ipykernel_3561/1800999524.py:46: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if vehicle_id is not -1:


AK6- 0.24208638072013855


KeyboardInterrupt: 

In [5]:
# OOP
df_cols = ['fr_number','v_id', 'v_bbox','lp_bbox','lp_bbox_score','lp','lp_score']
df_rows = []
# load using Sort
video_capture = cv2.VideoCapture(testvideo_path)

vehicle_detection = VehicleDetection('YOLOv8')
lp_detection = LicensePlateDetection('yolov8n_90e_cust')
lp_reader = LicensePlateReader('easyocr')

mot_trckr = Sort()
#results = {}
ret = True
frame_indexer=0
while ret:
    if (video_capture.isOpened()== False):
        print("Error opening video stream or file")
    
    ret, frame = video_capture.read()
    
    if ret:
        #if frame_indexer>10:
        #    break
        #results[frame_indexer] = {}
        #det
        det_objs = vehicle_detection.Detect(frame)
        vehicles=[]
        for det_obj in det_objs.boxes.data.tolist():
            obj_x1, obj_y1, obj_x2, obj_y2, obj_score, obj_class_id = det_obj
            if int(obj_class_id) in vehicle_detection.vehicle_ids:
                vehicles.append([obj_x1, obj_y1, obj_x2, obj_y2, obj_score])
                #print(obj_class_id)
        
        track_ids = mot_trckr.update(np.asarray(vehicles))
        #print(track_ids)
        #plates=[]
        det_lps = lp_detection.Detect(frame)
        for det_lp in det_lps.boxes.data.tolist():
            lp_x1, lp_y1, lp_x2, lp_y2, lp_score, lp_class_id = det_lp
            
            #assign license plate to vehicle
            vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2,vehicle_id = PlateToVehicle(det_lp,track_ids)
            if vehicle_id != -1:
                #crop
                license_plate = frame[int(lp_y1):int(lp_y2),int(lp_x1):int(lp_x2),:]
                lp_text, lp_text_confscore, lp_prep = lp_reader.MultiplyPrepAndRead(license_plate)

                #cv2.imshow('lp',license_plate)
                #cv2.waitKey(0)

                #cv2.imshow('lp',lp_psrep)
                #print(lp_text, lp_text_confscore)
                #cv2.waitKey(0)

                if lp_text != None:
                    df_rows.append({'fr_number':frame_indexer,
                                    'v_id':int(vehicle_id+1),
                                    'v_bbox':[vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2],
                                    'lp_bbox':[lp_x1, lp_y1, lp_x2, lp_y2],
                                    'lp_bbox_score':lp_score,
                                    'lp':lp_text,
                                    'lp_score':lp_text_confscore})
                
            
    
    frame_indexer+=1

[ WARN:0@415.320] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@415.320] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin2 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@415.320] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@415.320] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_

Error: results_list is empty or contains tuples without a second element.



0: 480x640 18 cars, 2 buss, 2 trucks, 92.5ms
Speed: 3.0ms preprocess, 92.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)



Error: results_list is empty or contains tuples without a second element.


0: 480x640 1 license, 98.7ms
Speed: 3.2ms preprocess, 98.7ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 18 cars, 2 buss, 2 trucks, 80.7ms
Speed: 2.8ms preprocess, 80.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 license, 91.6ms
Speed: 2.6ms preprocess, 91.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 18 cars, 2 buss, 2 trucks, 88.0ms
Speed: 2.4ms preprocess, 88.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 license, 91.6ms
Speed: 3.4ms preprocess, 91.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 18 cars, 2 buss, 2 trucks, 87.9ms
Speed: 2.6ms preprocess, 87.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 license, 94.3ms
Speed: 3.2ms preprocess, 94.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 18 cars, 2 buss, 2 trucks, 95.3ms
Speed: 2.9ms prepr

...


0: 480x640 13 cars, 1 bus, 2 trucks, 79.4ms
Speed: 2.2ms preprocess, 79.4ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 licenses, 74.3ms
Speed: 2.5ms preprocess, 74.3ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 15 cars, 1 bus, 2 trucks, 77.8ms
Speed: 2.3ms preprocess, 77.8ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 license, 76.0ms
Speed: 2.7ms preprocess, 76.0ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 15 cars, 1 bus, 2 trucks, 79.5ms
Speed: 3.1ms preprocess, 79.5ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 license, 84.1ms
Speed: 2.7ms preprocess, 84.1ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 14 cars, 1 bus, 3 trucks, 93.5ms
Speed: 2.9ms preprocess, 93.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 licenses, 87.5ms
Speed: 3.4ms prepro

Error: results_list is empty or contains tuples without a second element.



0: 480x640 14 cars, 1 bus, 2 trucks, 78.2ms
Speed: 2.5ms preprocess, 78.2ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 licenses, 79.8ms
Speed: 2.4ms preprocess, 79.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)


Error: results_list is empty or contains tuples without a second element.



0: 480x640 13 cars, 1 bus, 2 trucks, 76.7ms
Speed: 2.3ms preprocess, 76.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 license, 74.7ms
Speed: 3.1ms preprocess, 74.7ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 13 cars, 1 bus, 3 trucks, 77.9ms
Speed: 2.4ms preprocess, 77.9ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 76.0ms
Speed: 2.4ms preprocess, 76.0ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 13 cars, 1 bus, 3 trucks, 78.5ms
Speed: 3.0ms preprocess, 78.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 90.0ms
Speed: 2.7ms preprocess, 90.0ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 14 cars, 1 bus, 2 trucks, 97.2ms
Speed: 3.2ms preprocess, 97.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 licenses, 86.1ms
Speed: 2

In [6]:
out = pd.DataFrame(df_rows,columns=df_cols)
out.to_csv('/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/logs/log.csv')